In [1]:
import cv2
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
import shutil

In [2]:
# define a function to parse annotations 
def parse_annotation(directory):
    all_annotations = []
    
    # List all text files in the directory
    annotation_files = sorted([file for file in os.listdir(directory) if file.endswith(".txt")])
    
    for annotation_file in annotation_files:
        with open(os.path.join(directory, annotation_file), 'r') as file:
            lines = file.readlines()

        annotations = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # Skip lines with insufficient data

            class_id = int(parts[0])  # Class ID (integer)
            center_x = float(parts[1])  # X-coordinate of the bounding box center (float)
            center_y = float(parts[2])  # Y-coordinate of the bounding box center (float)
            width = float(parts[3])  # Width of the bounding box (float)
            height = float(parts[4])  # Height of the bounding box (float)

            annotation = [class_id, center_x, center_y, width, height]
            annotations.append(annotation)

        all_annotations.append(annotations)

    return all_annotations

In [3]:
# define a function to draw bounding boxes
def draw_bounding_boxes(image, annotations):
    for annotation in annotations:
        class_id = annotation[0]
        center_x = annotation[1]
        center_y = annotation[2]
        width = annotation[3]
        height = annotation[4]
        
        # Calculate bounding box coordinates
        x = int((center_x - width / 2) * image.shape[1])
        y = int((center_y - height / 2) * image.shape[0])
        w = int(width * image.shape[1])
        h = int(height * image.shape[0])
        
        # Draw the bounding box
        color = (255, 0, 0)  # Red
        thickness = 2
        cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
    
    return image


In [ ]:
# Get list of annotations
parsed_annotations_list = parse_annotation("data/labels")

# Get a list of image paths
image_dir = "/home/ankit/Data_Science/CV_Projects/OrderStack/data/images"
image_paths = sorted([os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith('.jpg')])
print(len(image_paths))

In [ ]:
# Get list of annotation paths
annotation_dir = "/home/ankit/Data_Science/CV_Projects/OrderStack/data/labels"
annotation_paths = sorted([os.path.join(annotation_dir, filename.replace('.jpg', '.txt')) for filename in os.listdir(image_dir) if filename.endswith('.jpg')])
print(len(annotation_paths))

In [ ]:
# # visulaise the images with bboxes using parsed annotations
# for image_path, parsed_annotations in zip(image_paths, parsed_annotations_list):

#     # load the image
#     image = cv2.imread(image_path)

#      # Draw bounding boxes on the image
#     image_with_boxes = draw_bounding_boxes(image=image, annotations=parsed_annotations)

#      # Display the image with bounding boxes using Matplotlib
#     plt.imshow(image_with_boxes)
#     plt.axis('off')
#     plt.show()

### Data Prep for Training


In [ ]:
# splitting into train test split
train_image_paths, val_image_paths, train_annotation_paths, val_annotation_paths = train_test_split(
    image_paths, annotation_paths, test_size=0.2, random_state=7)

# splitting the data into val and test set
val_image_paths, test_image_paths, val_annotation_paths, test_annotation_paths = train_test_split(
    val_image_paths, val_annotation_paths, test_size=0.5, random_state=7)

In [ ]:
# define a function to move the file to folders

def move_to_folder(list_of_files, destination):
    for f in list_of_files:
        try:
            shutil.copy(f, destination)

        except:
            print(f)
            assert False

In [ ]:
# # move the train and val set to folders
# move_to_folder(train_image_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/images/train')
# move_to_folder(train_annotation_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/labels/train')

# move_to_folder(val_image_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/images/val')
# move_to_folder(val_annotation_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/labels/val')

# move_to_folder(test_image_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/images/test')
# move_to_folder(test_annotation_paths, '/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/labels/test')

### Model Training

In [7]:
from ultralytics import YOLO

In [ ]:
# model = YOLO("yolov8n.yaml")
# model.train(data="./config.yaml", epochs=10)  # train the model

### Model testing

In [29]:
test_image_dir = "/home/ankit/Data_Science/CV_Projects/OrderStack/data_1/images/test"
test_image_paths = sorted([os.path.join(test_image_dir, filename) for filename in os.listdir(test_image_dir) if filename.endswith('.jpg')])
print(len(test_image_paths))

12


In [27]:
# load trained model
trained_model = YOLO('./runs/detect/train4/weights/best.pt')

In [30]:
# make prediction
results = trained_model(test_image_paths)  # return a list of Results objects


0: 640x640 7 class_0s, 1 class_1, 1 class_2, 1: 640x640 14 class_0s, 1 class_1, 1 class_2, 2: 640x640 14 class_0s, 1 class_1, 1 class_2, 3: 640x640 8 class_0s, 1 class_1, 1 class_2, 4: 640x640 9 class_0s, 1 class_1, 1 class_2, 5: 640x640 6 class_0s, 1 class_1, 2 class_2s, 6: 640x640 12 class_0s, 1 class_1, 1 class_2, 7: 640x640 12 class_0s, 1 class_1, 1 class_2, 8: 640x640 12 class_0s, 1 class_1, 1 class_2, 9: 640x640 12 class_0s, 2 class_1s, 1 class_2, 10: 640x640 9 class_0s, 1 class_1, 1 class_2, 11: 640x640 10 class_0s, 1 class_1, 1 class_2, 1503.7ms
Speed: 7.5ms preprocess, 125.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [32]:
# Run inference on image with arguments
trained_model.predict(test_image_paths, save=True, imgsz=320, conf=0.5)


0: 320x320 7 class_0s, 1 class_1, 1 class_2, 1: 320x320 13 class_0s, 1 class_1, 1 class_2, 2: 320x320 15 class_0s, 1 class_1, 1 class_2, 3: 320x320 8 class_0s, 1 class_1, 1 class_2, 4: 320x320 6 class_0s, 1 class_1, 5: 320x320 5 class_0s, 1 class_2, 6: 320x320 12 class_0s, 1 class_1, 1 class_2, 7: 320x320 12 class_0s, 1 class_1, 1 class_2, 8: 320x320 12 class_0s, 1 class_1, 1 class_2, 9: 320x320 10 class_0s, 1 class_2, 10: 320x320 9 class_0s, 1 class_1, 1 class_2, 11: 320x320 9 class_0s, 1 class_1, 1 class_2, 300.4ms
Speed: 1.3ms preprocess, 25.0ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)
Results saved to runs/detect/predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'class_0', 1: 'class_1', 2: 'class_2'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,


In [20]:

for result in results:
    boxes = result.boxes
    print(boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2., 0., 0., 0., 0., 0., 0., 1., 0.])
conf: tensor([0.9460, 0.9085, 0.9071, 0.9070, 0.8913, 0.8896, 0.8739, 0.8474, 0.8299])
data: tensor([[9.7442e+01, 1.2093e+02, 2.2268e+03, 1.2235e+03, 9.4603e-01, 2.0000e+00],
        [1.4443e+03, 1.2906e+02, 1.7078e+03, 1.1971e+03, 9.0852e-01, 0.0000e+00],
        [7.3034e+01, 1.2478e+02, 7.2022e+02, 1.2132e+03, 9.0713e-01, 0.0000e+00],
        [1.1923e+03, 1.3395e+02, 1.4475e+03, 1.2134e+03, 9.0702e-01, 0.0000e+00],
        [8.9742e+02, 1.3171e+02, 1.1992e+03, 1.2148e+03, 8.9127e-01, 0.0000e+00],
        [1.9696e+03, 1.2974e+02, 2.2381e+03, 1.1972e+03, 8.8956e-01, 0.0000e+00],
        [1.7066e+03, 1.3212e+02, 1.9567e+03, 1.2096e+03, 8.7388e-01, 0.0000e+00],
        [1.1364e+02, 1.3324e+02, 2.2544e+03, 2.7448e+02, 8.4737e-01, 1.0000e+00],
        [7.2073e+02, 1.3357e+02, 8.9763e+02, 1.2139e+03, 8.2987e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (3368, 2380)
shape: tor

In [ ]:
# visulaise the images with bboxes using parsed annotations

# load the image
image = cv2.imread(test_image_path)

    # Draw bounding boxes on the image
image_with_boxes = draw_bounding_boxes(image=image, annotations=parsed_annotations)

    # Display the image with bounding boxes using Matplotlib
plt.imshow(image_with_boxes)
plt.axis('off')
plt.show()